In [1]:
import pandas as pd
import numpy as np
from data_analysis import get_heart_bounds, correlation_coef, graph_3d, get_index
from graph import *
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer
# from prettytable import PrettyTable

def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))


In [2]:
ecgs = pd.read_csv("new_simu-data/Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv("new_simu-data/Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv("new_simu-data/UVC/Coord1_UVC_4000.csv", header=None).to_numpy()
bounds = get_heart_bounds(labels) # getting bounds 
tidx = np.random.randint(0, labels.shape[0])    # Pick out a sample to use as a target
target, target_ecg,target_uvc = labels[tidx], ecgs[tidx], uvc[tidx]
print("Target: ", target)
init = 5 #initial random sample
steps = 5 # total AL step
af = "ucb" 


Target:  [  7.81502 -93.7713   11.8206 ]


In [3]:
# labels = uvc[:,0:3]
# bounds = get_heart_bounds(labels) # getting bounds 
# print("Target: ", target_uvc)

optimizer = BayesianOptimization(f=black_box,pbounds=bounds,random_state=None, real_set=labels)
gp,X = optimizer.gpfit(init_points=init, n_iter=steps,  acq=af, kappa = 1,kappa_decay=0.75,kappa_decay_delay=2)
#     heart(target,labels)



|   iter    |  target   |     x     |     y     |     z     |
-------------------------------------------------------------
|  1        |  0.1075   | -15.99    | -20.29    | -18.29    |
|  2        |  0.11     | -28.54    | -52.4     | -1.488    |
|  3        |  0.3784   |  33.28    | -44.5     | -35.71    |
|  4        |  0.7813   | -15.33    | -101.1    |  23.56    |
|  5        |  0.2011   | -31.17    | -43.67    | -15.66    |
|  6        |  0.7693   | -15.79    | -100.9    |  21.56    |
|  7        |  0.8937   |  1.881    | -98.24    |  23.07    |
|  8        |  0.9058   |  0.1864   | -98.44    |  21.75    |
|  9        |  0.9108   |  2.601    | -103.5    |  11.29    |
|  10       |  0.718    |  20.0     | -96.78    |  11.09    |


In [4]:
def test(init,target,target_ecg,labels,ecgs,visited,predicted):
    """
    show the points and path of both predicted and visited values
    """
    color_gradient = []
    for ecg, coord in zip(ecgs, labels):
        cc = correlation_coef(target_ecg, ecg)
        color_gradient.append(cc)
        
    path_pred = np.array(predicted)
    path = np.array(visited)
    path1 = path[0:init,:]
    path2 = path[init:len(path),:]
    
#     rest = np.delete(labels, np.where(np.isin(labels, path2)), axis=0)
#     color_gradient = np.delete(color_gradient, np.where(np.isin(labels, path2)), axis=0)
    fig = plt.figure(figsize=(10,7))
    ax = fig.gca(projection='3d')
    img = ax.scatter(xs=labels[:, 0], ys=labels[:, 1], zs=labels[:, 2], zdir='z', alpha=0.75,s = 5, c=color_gradient, cmap = plt.cm.rainbow)
    ax.scatter(xs=path2[:, 0], ys=path2[:, 1], zs=path2[:, 2], zdir='z',s=20, color='blue')
    ax.scatter(xs=path_pred[:, 0], ys=path_pred[:, 1], zs=path_pred[:, 2], zdir='z',s=20, color='green')
    ax.plot(path2[:, 0], path2[:, 1], path2[:, 2], color = 'black',label = 'Visited Path')
    ax.plot(path_pred[:, 0], path_pred[:, 1], path_pred[:, 2], color = 'red',label='Predicted Path')
    ax.legend()
    fig.colorbar(img)
    m = path2
    n = path_pred
    for i in range(len(m)):
        ax.text(m[i, 0], m[i, 1], m[i, 2], '%s' % (str(i+init+1)), size=10, zorder=1, color='black')
        ax.text(n[i, 0], n[i, 1], n[i, 2], '%s' % (str(i+init+1)), size=10, zorder=1, color='red')
    ax.scatter(xs=target[0], ys=target[1], zs=target[2], color='black',marker = "*", s = 150)
    fig.suptitle('Path of BO to target', fontsize=16)
    plt.show()
#     anim = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), repeat=True, fargs=(fig, ax))
#     anim.save('plots/AL_path'+str(labels[1])+'.gif', dpi=80, writer='imagemagick', fps=10)

In [6]:
# graph_cc_distribution(target_ecg,ecgs,labels)
# gp_plot(gp,labels,target_uvc)
test(init,target,target_ecg,labels,ecgs,optimizer.visited,optimizer.predicted)

In [ ]:
ecgs.shape